In [1]:
from textwrap import dedent
from typing import AsyncGenerator, List, Optional

# Import our agent team
from app.agents.a_problem_definition.problem_validator import ProblemValidatorFeedback, problem_validator_prompt_instructions
from app.agents.b_initial_research import create_market_research, create_competitor_analysis, create_customer_insights, create_online_trends
from app.agents.c_research_reviewer import create_research_reviewer
from app.agents.d_feasibility_research import create_finance_feasibility, create_operations_feasibility, create_tech_feasibility
from app.agents.e_strategy_research import create_go_to_market, create_monetization, create_risk_analysis
from app.agents.f_output_production import create_landing_page_poc, create_podcaster, create_summarizer

from app.workflows.single import AgentRunEvent, AgentRunResult, FunctionCallingAgent
from llama_index.core.chat_engine.types import ChatMessage
from llama_index.core.prompts import PromptTemplate
from llama_index.core.settings import Settings
from llama_index.core.workflow import (
    Context,
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

from app.agents.events import QnaWorkflowEvent, RiskAnalysisCompleteEvent, GetRiskAnalysisCritiqueEvent, SummarizeEverythingEvent, ResearchWorkflowEvent, MarketResearchFeedbackEvent, CustomerInsightsFeedbackEvent, OnlineTrendsFeedbackEvent, CompetitorAnalysisFeedbackEvent, InitialResearchCompleteEvent, GetMarketResearchCritiqueEvent, GetCustomerInsightsCritiqueEvent, GetOnlineTrendsCritiqueEvent, GetCompetitorAnalysisCritiqueEvent, StartResearchPipelineEvent

In [2]:
# Add at the beginning of your notebook
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [3]:
from app.settings import init_openai


init_openai()

In [4]:
competitor_analyst = create_competitor_analysis(chat_history=[])
print([tool.metadata for tool in competitor_analyst.tools])

[ToolMetadata(description="tavily_search(query: str, search_depth: str = 'basic', max_results: int = 5, api_key: Optional[str] = None)\n\n    Use this function to search for any query using Tavily's API.\n    Args:\n        query (str): The query to search.\n        search_depth (str): The search depth to use ('basic' or 'advanced'). Default is 'basic'.\n        max_results (int): The maximum number of results to be returned. Default is 10.\n        api_key (Optional[str]): Tavily API key. If not provided, will look for TAVILY_API_KEY env variable.\n    ", name='tavily_search', fn_schema=<class 'llama_index.core.tools.utils.tavily_search'>, return_direct=False), ToolMetadata(description="tavily_qna_search(query: str, api_key: Optional[str] = None)\n\n    Use this function to get quick answers to questions using Tavily's API.\n    Args:\n        query (str): The question to ask.\n        api_key (Optional[str]): Tavily API key. If not provided, will look for TAVILY_API_KEY env variable.

In [5]:
handler = competitor_analyst.run(input="""
    Product idea: Robot vacuum cleaner for robot cars
    Problem statement: So many people are using robot cars, but no one is using robot vacuum cleaners for them.
""")
async for event in handler.stream_events():
    if isinstance(event, AgentRunEvent):
        print(event.msg)
result = await handler
result

Start to work on: 
    Product idea: Robot vacuum cleaner for robot cars
    Problem statement: So many people are using robot cars, but no one is using robot vacuum cleaners for them.

Calling tools: [ToolSelection(tool_id='call_G7abgiQW11KgZCaAvIFsUfyi', tool_name='tavily_search', tool_kwargs={'query': 'robot vacuum cleaner for robot cars startup', 'search_depth': 'advanced', 'max_results': 5})]
Tool response: tool: {'query': 'robot vacuum cleaner for robot cars startup', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'The 7 Best Robot Vacuums of 2024 - Robot Vacuum Reviews - Popular Mechanics', 'url': 'https://www.popularmechanics.com/home/interior-projects/a32892415/robot-vacuum-reviews/', 'content': 'The Best Robot Vacuums. Best Overall: Roborock Q5+ Best to Vacuum-Mop Combo: iRobot Roomba Combo j7+ Best Value: Eufy RoboVac G30 Edge Robot Vacuum Lowest Maintenance: iRobot Roomba s9+ Robot', 'score': 0.92563426, 'raw_content': None}, {'title': 'Vol

AgentRunResult(response=ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='### Competitive Landscape Analysis for Robot Vacuum Cleaner for Robot Cars\n\n#### Competitor 1: Volkswagen ID. Code Concept with Integrated Robot Vacuum Cleaner\n\n1. **Core Solution Analysis**\n   - **Main Features and Functionality**: The Volkswagen ID. Code concept features an integrated robot vacuum cleaner named "Lupo" that operates autonomously to clean the vehicle\'s interior. It includes advanced driving assistance systems for Level 4 automated driving.\n   - **Key Differentiators**: The integration of a robot vacuum within a concept electric SUV, showcasing a futuristic design and advanced AI capabilities.\n   - **Target Audience and Use Cases**: Targeted at tech-savvy consumers interested in electric vehicles and autonomous driving, particularly in urban environments.\n   - **Pricing Structure**: As a concept vehicle, pricing is not available, but it is expected to be

In [6]:
print(result)


response=ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='### Competitive Landscape Analysis for Robot Vacuum Cleaner for Robot Cars\n\n#### Competitor 1: Volkswagen ID. Code Concept with Integrated Robot Vacuum Cleaner\n\n1. **Core Solution Analysis**\n   - **Main Features and Functionality**: The Volkswagen ID. Code concept features an integrated robot vacuum cleaner named "Lupo" that operates autonomously to clean the vehicle\'s interior. It includes advanced driving assistance systems for Level 4 automated driving.\n   - **Key Differentiators**: The integration of a robot vacuum within a concept electric SUV, showcasing a futuristic design and advanced AI capabilities.\n   - **Target Audience and Use Cases**: Targeted at tech-savvy consumers interested in electric vehicles and autonomous driving, particularly in urban environments.\n   - **Pricing Structure**: As a concept vehicle, pricing is not available, but it is expected to be premium due to

In [30]:
import random

class IdeaResearchWorkflow(Workflow):
    def __init__(
        self, timeout: int = 600, chat_history: Optional[List[ChatMessage]] = None
    ):
        super().__init__(timeout=timeout)
        self.chat_history = chat_history or []

    @step()
    async def start(self, ctx: Context, ev: StartEvent) -> QnaWorkflowEvent | ResearchWorkflowEvent:
        # set streaming
        ctx.data["streaming"] = getattr(ev, "streaming", False)
        # start the workflow with researching about a topic
        ctx.data["task"] = ev.input
        ctx.data["user_input"] = ev.input

        # Decision-making process
        prompt_template = PromptTemplate(
            dedent(
                """
                ### High Level Context
                You are part of a workflow to help users do autonomous research and idea validation for their business ideas.

                ### Your Role
                You are an expert in decision-making, given the chat history and the new user request, decide whether the user request is a research query (including providing more information for an existing research query) or a question for the AI.

                Here is the chat history:
                {chat_history}

                The current user request is:
                {input}

                Decision (respond with either 'research' or 'qna'):
            """
            )
        )

        chat_history_str = "\n".join(
            [f"{msg.role}: {msg.content}" for msg in self.chat_history]
        )
        prompt = prompt_template.format(chat_history=chat_history_str, input=ev.input)

        output = await Settings.llm.acomplete(prompt)
        decision = output.text.strip().lower()

        if decision == "research":
            return ResearchWorkflowEvent(input=f"User input: {ev.input}")
        else:
            return QnaWorkflowEvent(input=f"User input: {ev.input}")

    @step()
    async def validate_problem_statement(self, ctx: Context, ev: ResearchWorkflowEvent) -> StartResearchPipelineEvent | StopEvent:
        chat_history_str = "\n".join(
            [f"{msg.role}: {msg.content}" for msg in self.chat_history]
        )
        prompt = problem_validator_prompt_instructions.format(chat_history=chat_history_str, input=ev.input)

        output = await Settings.llm.as_structured_llm(output_cls=ProblemValidatorFeedback).acomplete(prompt)
        result: ProblemValidatorFeedback = output.raw
        if result.enough_information:
            return StartResearchPipelineEvent(input=result.refined_problem_statement)
        else:
            return StopEvent(input=result.feedback)

    @step()
    async def qna(self, ctx: Context, ev: QnaWorkflowEvent) -> StopEvent:
        messages = self.chat_history + [ChatMessage(role="user", content=ev.input)]
        result = await Settings.llm.achat(messages)
        return StopEvent(result=result.content)
    
    ### Initial Research Analysts Team 1 ###

    ### Market Research ###
    @step()
    async def market_research(self, ctx: Context, ev: StartResearchPipelineEvent | MarketResearchFeedbackEvent, market_research_agent: FunctionCallingAgent) -> GetMarketResearchCritiqueEvent:
        result: AgentRunResult = await self.run_agent(ctx, market_research_agent, ev.input)
        content = result.response.message.content
        return GetMarketResearchCritiqueEvent(input=content)

    @step()
    async def critique_market_research(self, ctx: Context, ev: GetMarketResearchCritiqueEvent, market_research_critic_agent: FunctionCallingAgent) -> InitialResearchCompleteEvent | MarketResearchFeedbackEvent:
        result: AgentRunResult = await self.run_agent(ctx, market_research_critic_agent, ev.input)
        content = result.response.message.content
        if random.random() > 0.5:
            return InitialResearchCompleteEvent(input=content)
        else:
            return MarketResearchFeedbackEvent(input=content)

    ### Customer Insights ###
    @step()
    async def customer_insights(self, ctx: Context, ev: StartResearchPipelineEvent | CustomerInsightsFeedbackEvent, customer_insights_agent: FunctionCallingAgent) -> GetCustomerInsightsCritiqueEvent:
        result: AgentRunResult = await self.run_agent(ctx, customer_insights_agent, ev.input)
        content = result.response.message.content
        return GetCustomerInsightsCritiqueEvent(input=content)

    @step()
    async def critique_customer_insights(self, ctx: Context, ev: GetCustomerInsightsCritiqueEvent, customer_insights_critic_agent: FunctionCallingAgent) -> InitialResearchCompleteEvent | CustomerInsightsFeedbackEvent:
        result: AgentRunResult = await self.run_agent(ctx, customer_insights_critic_agent, ev.input)
        content = result.response.message.content
        if random.random() > 0.5:
            return InitialResearchCompleteEvent(input=content)
        else:
            return CustomerInsightsFeedbackEvent(input=ev.input)    

    ### Online Trends ###
    @step()
    async def online_trends(self, ctx: Context, ev: StartResearchPipelineEvent | OnlineTrendsFeedbackEvent, online_trends_agent: FunctionCallingAgent) -> GetOnlineTrendsCritiqueEvent:
        result: AgentRunResult = await self.run_agent(ctx, online_trends_agent, ev.input)
        content = result.response.message.content
        return GetOnlineTrendsCritiqueEvent(input=content)

    @step()
    async def critique_online_trends(self, ctx: Context, ev: GetOnlineTrendsCritiqueEvent, online_trends_critic_agent: FunctionCallingAgent) -> InitialResearchCompleteEvent | OnlineTrendsFeedbackEvent:
        result: AgentRunResult = await self.run_agent(ctx, online_trends_critic_agent, ev.input)
        content = result.response.message.content
        if random.random() > 0.5:
            return InitialResearchCompleteEvent(input=content)
        else:
            return OnlineTrendsFeedbackEvent(input=ev.input)

    ### Competitor Analysis ###
    @step()
    async def competitor_analysis(self, ctx: Context, ev: StartResearchPipelineEvent | CompetitorAnalysisFeedbackEvent, competitor_analysis_agent: FunctionCallingAgent) -> GetCompetitorAnalysisCritiqueEvent:
        result: AgentRunResult = await self.run_agent(ctx, competitor_analysis_agent, ev.input)
        content = result.response.message.content
        return GetCompetitorAnalysisCritiqueEvent(input=content)

    @step()
    async def critique_competitor_analysis(self, ctx: Context, ev: GetCompetitorAnalysisCritiqueEvent, competitor_analysis_critic_agent: FunctionCallingAgent) -> InitialResearchCompleteEvent | CompetitorAnalysisFeedbackEvent:
        result: AgentRunResult = await self.run_agent(ctx, competitor_analysis_critic_agent, ev.input)
        content = result.response.message.content
        if random.random() > 0.5:
            return InitialResearchCompleteEvent(input=content)
        else:
            return CompetitorAnalysisFeedbackEvent(input=ev.input)

    ### Collect Initial Research Feedback ###
    @step()
    async def review_initial_research(self, ctx: Context, ev: InitialResearchCompleteEvent, research_reviewer_agent: FunctionCallingAgent) -> SummarizeEverythingEvent:
        print("Received event ", ev.result)

        # wait until we receive all 4 prior events
        if (
            ctx.collect_events(
                ev,
                [InitialResearchCompleteEvent] * 4,
            )
            is None
        ):
            return None

        result: AgentRunResult = await self.run_agent(ctx, research_reviewer_agent, ev.input)
        content = result.response.message.content
        return SummarizeEverythingEvent(input=content)

    ### Output Production ###
    @step()
    async def summarize_everything(self, ctx: Context, ev: SummarizeEverythingEvent, summarizer_agent: FunctionCallingAgent) -> StopEvent:
        result: AgentRunResult = await self.run_agent(ctx, summarizer_agent, ev.input)
        content = result.response.message.content
        return StopEvent(result=content)

    async def run_agent(
        self,
        ctx: Context,
        agent: FunctionCallingAgent,
        input: str,
        streaming: bool = False,
    ) -> AgentRunResult | AsyncGenerator:
        handler = agent.run(input=input, streaming=streaming)
        # bubble all events while running the executor to the planner
        async for event in handler.stream_events():
            # Don't write the StopEvent from sub task to the stream
            if type(event) is not StopEvent:
                ctx.write_event_to_stream(event)
        return await handler

In [7]:
from app.workflows.single import FunctionCallingAgent
from llama_index.utils.workflow import draw_all_possible_flows
draw_all_possible_flows(FunctionCallingAgent)

<class 'NoneType'>
<class 'app.workflows.single.ToolCallEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class 'app.workflows.single.InputEvent'>
<class 'app.workflows.single.InputEvent'>
workflow_all_flows.html


In [31]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(IdeaResearchWorkflow)

<class 'NoneType'>
<class '__main__.GetCompetitorAnalysisCritiqueEvent'>
<class '__main__.InitialResearchCompleteEvent'>
<class '__main__.CompetitorAnalysisFeedbackEvent'>
<class '__main__.InitialResearchCompleteEvent'>
<class '__main__.CustomerInsightsFeedbackEvent'>
<class '__main__.InitialResearchCompleteEvent'>
<class '__main__.MarketResearchFeedbackEvent'>
<class '__main__.InitialResearchCompleteEvent'>
<class '__main__.OnlineTrendsFeedbackEvent'>
<class '__main__.GetCustomerInsightsCritiqueEvent'>
<class '__main__.GetMarketResearchCritiqueEvent'>
<class '__main__.GetOnlineTrendsCritiqueEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.SummarizeEverythingEvent'>
<class '__main__.QnaWorkflowEvent'>
<class '__main__.ResearchWorkflowEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.StartResearchPipelineEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>


workflow_all_flows.html


In [7]:
workflow = IdeaResearchWorkflow(timeout=360)
result = await workflow.run(input="What's LlamaIndex?")
result

WorkflowValidationError: The following events are produced but never consumed: StartFeasibilityResearchEvent, MarketResearchFeedbackEvent

In [6]:
from llama_index.core.workflow import (
    Context,
    Event,
    StartEvent,
    InputRequiredEvent,
    HumanResponseEvent,
    StopEvent,
    Workflow,
    step,
)

In [28]:
API_KEY = ""
from llama_index.llms.openai import OpenAI
from typing import List, Optional
from llama_index.core.settings import Settings
from app.agents.new_workflow import StartResearchPipelineEvent
from llama_cloud import ChatMessage
from app.agents.a_problem_definition.problem_validator import problem_definer_prompt_instructions, ProblemDefinerFeedback

class TestEvent(Event):
    input: str

class OpenAIGenerator(Workflow):
    def __init__(
        self, timeout: int = 600, chat_history: Optional[List[ChatMessage]] = None
    ):
        super().__init__(timeout=timeout)
        self.chat_history = chat_history or []
        
    @step
    async def generate(self, ev: StartEvent) -> TestEvent | StartResearchPipelineEvent:
        # Validate user problem statement
        prompt_template = problem_definer_prompt_instructions
        chat_history_str = "\n".join(
            [f"{msg.role}: {msg.content}" for msg in self.chat_history]
        )
        prompt = prompt_template.format(chat_history=chat_history_str, input="TEST")

        output = await OpenAI(model="gpt-4o-mini", api_key=API_KEY).as_structured_llm(output_cls=ProblemDefinerFeedback).acomplete(prompt)
        res: ProblemDefinerFeedback = output.raw
        
        if res.enough_information:
            return StartResearchPipelineEvent(input=res.refined_problem_statement)
        else:
            return TestEvent(input=res.feedback)
        
    @step
    async def step2(self, ev: TestEvent) -> StopEvent:
        return StopEvent(result=ev.input)
    
    @step
    async def step3(self, ev: StartResearchPipelineEvent) -> StopEvent:
        return StopEvent(result=ev.input)


w = OpenAIGenerator(timeout=10)
handler = w.run(query="What's LlamaIndex?")
async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        # here, we can handle human input however you want
        # this means using input(), websockets, accessing async state, etc.
        # here, we just use input()
        response = input(event.prefix)
        handler.ctx.send_event(HumanResponseEvent(response=response))

await handler

In [42]:
from typing import List, Optional
from llama_index.core.settings import Settings
from app.agents.new_workflow import StartResearchPipelineEvent
from llama_cloud import ChatMessage
from app.agents.a_problem_definition.problem_validator import problem_definer_prompt_instructions, ProblemDefinerFeedback

class IdeaResearcdWorkflow(Workflow):
    def __init__(
        self, timeout: int = 600, chat_history: Optional[List[ChatMessage]] = None
    ):
        super().__init__(timeout=timeout)
        self.chat_history = chat_history or []
        
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> TestEvent | StartResearchPipelineEvent:
        # Validate user problem statement
        prompt_template = problem_definer_prompt_instructions
        chat_history_str = "\n".join(
            [f"{msg.role}: {msg.content}" for msg in self.chat_history]
        )
        prompt = prompt_template.format(chat_history=chat_history_str, input="TEST")

        output = await OpenAI(model="gpt-4o-mini", api_key=API_KEY).as_structured_llm(output_cls=ProblemDefinerFeedback).acomplete(prompt)
        res: ProblemDefinerFeedback = output.raw
        
        if res.enough_information:
            return StartResearchPipelineEvent(input=res.refined_problem_statement)
        else:
            return TestEvent(input=res.feedback)
        
    @step
    async def step2(self, ev: TestEvent) -> StopEvent:
        return StopEvent(result=ev.input)
    
    @step
    async def step3(self, ev: StartResearchPipelineEvent) -> StopEvent:
        return StopEvent(result=ev.input)

    
workflow = OpenAIGenerator(timeout=360)
result = await workflow.run(query="What's LlamaIndex?")
result



"The input 'TEST' is too vague and does not provide any specific information about a problem, the affected individuals, or the impact on their lives. Please provide more detailed information about the business idea or problem you want to address."